In [ ]:
%%HTML
<h1>Majdoubi's Guide to a beginner's first Kaggle Project (top 20%)</h1>
<h3>Made by : Ahmed Amine MAJDOUBI</h3>

In this notebook, we will go through the basic steps of making predictions based on a given dataset. The steps that I followed in this notebook are as follows :
- Importing Libraries and Datasets
- Data Description
- Finding Correlation Features
- Removing Outliers
- Imputation of missing values
- Correcting Features
- Adding Features
- Skewness and Kurtosis
- Label Encoding
- Transformation and Scaling
- Feature Selection
- Principal Component Analysis
- Testing Different Models
- Hyper-parameter Tuning
- Combining Models
- Making Predictions

In [ ]:
%%HTML
<h1>Importing Libraries and Datasets</h1>

In [ ]:
#importing librairies

import pandas as pd # data processing
import numpy as np # numeric computation
import matplotlib.pyplot as plt # plot visualization
%matplotlib inline
import seaborn as sns # data visualisation
color = sns.color_palette()
sns.set_style('darkgrid')
import scipy.stats as st # statistics
pd.options.display.max_columns = None # show all columns
import missingno as msno # missing data visualizations and utilities
import warnings # ignore file warnings
warnings.filterwarnings('ignore')

# Importing the train and test datasets in  pandas dataframe

trainData = pd.read_csv('../input/train.csv')
testData = pd.read_csv('../input/test.csv')
trainData.drop(columns = 'Id', inplace =True)
y_train = trainData['SalePrice']


In [ ]:
%%HTML
<h1>Data Description</h1>

First let's start by looking at the data and getting a general idea about it

In [ ]:
# shape of data
trainData.shape , testData.shape, y_train.shape

In [ ]:
# A basic description of the dataset
trainData.describe()

In [ ]:
# display the first five rows of the train dataset.
trainData.head()

In [ ]:
# display the last five rows of the train dataset.
trainData.tail()

In [ ]:
# Numeric and categorical features in the dataset
trainData.select_dtypes(include=[np.number]).columns, trainData.select_dtypes(include=[np.object]).columns

In [ ]:
%%HTML
<h1>Correlation Features</h1>

Let's look at some correlation features between features.

In [ ]:
# Showing the numerical varibales with the highest correlation with 'SalePrice', sorted from highest to lowest
correlation = trainData.select_dtypes(include=[np.number]).corr()
print(correlation['SalePrice'].sort_values(ascending = False))

In [ ]:
# Heatmap of correlation of numeric features
f , ax = plt.subplots(figsize = (14,12))
plt.title('Correlation of numeric features',size=15)
sns.heatmap(correlation,square = True,  vmax=0.8)

- We observe two white squares (2,2 and 3,3) in the heatmap indicating high correlation. The first group of highly correlated variables is 'TotalBsmtSF' and '1stFlrSF'. The second group is 'GarageYrBlt', 'GarageCars' and 'GarageArea'. This indicates the presence of multicollinearity.
- The other four white squares (1,1) just indicate an obvious correlation between 'GarageYrBlt' and 'YearBuilt' and between 'TotRmsAbvGrd' and 'GrLivArea'
- We also observe  from the heatmap and the previous correlation list that'GrLivArea', 'TotalBsmtSF', 'OverallQual', 'FullBath', 'TotRmsAbvGrd' and 'YearBuilt' are highly correlated with 'SalePrice'

In [ ]:
# Zoomed HeatMap of the most Correlayed variables
zoomedCorrelation = correlation.loc[['SalePrice','GrLivArea','TotalBsmtSF','OverallQual','FullBath','TotRmsAbvGrd','YearBuilt','1stFlrSF','GarageYrBlt','GarageCars','GarageArea'], ['SalePrice','GrLivArea','TotalBsmtSF','OverallQual','FullBath','TotRmsAbvGrd','YearBuilt','1stFlrSF','GarageYrBlt','GarageCars','GarageArea']]
f , ax = plt.subplots(figsize = (14,12))
plt.title('Correlation of numeric features',size=15)
sns.heatmap(zoomedCorrelation, square = True, linewidths=0.01, vmax=0.8, annot=True,cmap='viridis',
            linecolor="black", annot_kws = {'size':12})

We conclude that :
- 'TotalBsmtSF' and '1stFlrSF' are strongly correlated
- 'TotRmsAbvGrd' and 'GrLivArea' are strongly correlated
- 'GarageCars' and 'GarageArea' are strongly correlated
- 'GarageYrBlt' and 'YearBuilt' are strongly correlated
- 'TotRmsAbvGrd' and 'GrLivArea' are strongly correlated
- 'OverallQual', 'GrLivArea' and 'TotRmsAbvGrd' are strongly correlated with 'SalePrice'

In [ ]:
# Pair plot
sns.set()
cols = ['SalePrice','GrLivArea','TotalBsmtSF','OverallQual','FullBath','TotRmsAbvGrd','YearBuilt','1stFlrSF','GarageYrBlt','GarageCars','GarageArea']
sns.pairplot(trainData[cols],size = 2 ,kind ='scatter',diag_kind='kde')
plt.show()

- We observe that 'SalePrice' increases almost quadratically with 'TotalBsmtSF', 'GrLivArea', '1stFlrSF'. So we conclude that the price of the houses increases quadratically with its surface area. We also observe that 'SalePrice' increases exponentially with 'OverallQual'.
- We also observe from 'GrLivArea'-'1stFlSF' and '1stFlSF'-'TotalBsmSF' that all the points are above the identity fucntion line, which means that the ground living area has the biggest surface of all floors, and that the first floor area is generally bigger than the basement area.
- We observe the same phenomenon for 'GarageYrBlt'-'YearBuilt'. which makes sense since we start building the garage after building the house, altough there are some exceptions in the data.

In [ ]:
%%HTML
<h1>Removing Outliers</h1>

From the previous pair plots, we can see that there are outliers for 'TotalBsmtSF', '1stFlrSF' and 'GrLivArea'. Let's use the scatterplot to observe these outliers more precisely

In [ ]:
plt.figure(figsize=(7,5))
plt.scatter(x = trainData.TotalBsmtSF,y = trainData.SalePrice)
plt.title('TotalBsmtSF', size = 15)
plt.figure(figsize=(7,5))
plt.scatter(x = trainData['1stFlrSF'],y = trainData.SalePrice)
plt.title('1stFlrSF', size = 15)
plt.figure(figsize=(7,5))
plt.scatter(x = trainData.GrLivArea,y = trainData.SalePrice)
plt.title('GrLivArea', size = 15)

In [ ]:
# Removing the outliers
trainData.drop(trainData[trainData['TotalBsmtSF'] > 5000].index,inplace = True)
trainData.drop(trainData[trainData['1stFlrSF'] > 4000].index,inplace = True)
trainData.drop(trainData[(trainData['GrLivArea'] > 4000) & (trainData['SalePrice']<300000)].index,inplace = True)
trainData.shape

Since only two outliers were dropped, it means that the three features shared the same outlier 

In [ ]:
%%HTML
<h1>Imputation of missing values</h1>

Let's look at the missing valeus in our data. We will be using msno library. Msno provides a small toolset of flexible and easy-to-use missing data visualizations and utilities that allows you to get a quick visual summary of the completeness (or lack thereof) of your dataset

In [ ]:
# Visualising missing values of numeric features for sample of 200
msno.matrix(trainData.select_dtypes(include=[np.number]).sample(200))

In [ ]:
# Visualising percentage of missing values of the top 10 numeric variables
total = trainData.select_dtypes(include=[np.number]).isnull().sum().sort_values(ascending=False)
percent = (trainData.select_dtypes(include=[np.number]).isnull().sum()/trainData.select_dtypes(include=[np.number]).isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1,join='outer', keys=['Missing Count', 'Missing Percentage'])
missing_data.index.name =' Numeric Feature'
missing_data.head(10)

We observe that 'LotFrontage', 'GarageYrBlt' and 'MasVnrArea' are the only one who have missing values

In [ ]:
# Visualising missing values of categorical features for sample of 200
msno.matrix(trainData.select_dtypes(include=[np.object]).sample(200))

In [ ]:
# Visualising percentage of missing values of the top 10 categorical variables
total = trainData.select_dtypes(include=[np.object]).isnull().sum().sort_values(ascending=False)
percent = (trainData.select_dtypes(include=[np.object]).isnull().sum()/trainData.select_dtypes(include=[np.object]).isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1,join='outer', keys=['Missing Count', 'Missing Percentage'])
missing_data.index.name =' Numeric Feature'
missing_data.head(10)

We observe that 'PoolQC', 'MiscFeature', 'Alley', 'Fence' and 'FireplaceQu' have a significant amount of missing values (at least more than half of observation)

In [ ]:
# Visualization of nullity by column
msno.bar(trainData.sample(1000))

In [ ]:
# Nullity correlation heatmap : how strongly the presence or absence of one variable affects the presence of another
msno.heatmap(trainData)

# -1 : if one variable appears the other definitely does not
# 0 : variables appearing or not appearing have no effect on one another 
# 1 : if one variable appears the other definitely also does

In [ ]:
# Dendogram for variable completion, reveals trends deeper than the pairwise ones visible in the correlation heatmap
msno.dendrogram(trainData)

Cluster leaves which linked together at a distance of zero fully predict one another's presence : one variable might always be empty when another is filled, or they might always both be filled or both empty.

First of all, let's start by replacing the missing values in both the training and the test set. So we will be combining both datasets into one dataset

In [ ]:
# Concatenate the training and test datasets into a single datafram
dataFull = pd.concat([trainData,testData],ignore_index=True)
dataFull.drop('Id',axis = 1,inplace = True)
dataFull.shape

In [ ]:
# Sum of missing values by feature
sumMissingValues = dataFull.isnull().sum()
sumMissingValues[sumMissingValues>0].sort_values(ascending = False)

In [ ]:
# Numeric features : replace with 0
for col in ['BsmtFullBath','BsmtHalfBath','BsmtUnfSF','TotalBsmtSF','GarageCars','BsmtFinSF2','BsmtFinSF1','GarageArea']:
    dataFull[col].fillna(0,inplace= True)

# Check if missing values are imputed successfully
dataFull.isnull().sum()[dataFull.isnull().sum()>0].sort_values(ascending = False)

Let's start by imputing features with less than five missing values

In [ ]:
# Categorical features : replace with the mode (most frequently occured value)
for col in ['MSZoning','Functional','Utilities','KitchenQual','SaleType','Exterior2nd','Exterior1st','Electrical']:
    dataFull[col].fillna(dataFull[col].mode()[0],inplace= True)

# Check if missing values are imputed successfully
dataFull.isnull().sum()[dataFull.isnull().sum()>0].sort_values(ascending = False)

In [ ]:
# Impute features with more than five missing values

# Categorical features : replace all with 'None'
for col in ['PoolQC','MiscFeature','Alley','Fence','FireplaceQu','GarageQual','GarageCond','GarageFinish','GarageType','BsmtExposure','BsmtCond','BsmtQual','BsmtFinType2','BsmtFinType1','MasVnrType']:
    dataFull[col].fillna('None',inplace = True)

# Check if missing values are imputed successfully
dataFull.isnull().sum()[dataFull.isnull().sum()>0].sort_values(ascending = False)

Since 'MasVnrArea' only have 23 missing values, we can replace them with the mean of the column

In [ ]:
dataFull['MasVnrArea'].fillna(dataFull['MasVnrArea'].mean(), inplace=True)

# Check if missing values are imputed successfully
dataFull.isnull().sum()[dataFull.isnull().sum()>0].sort_values(ascending = False)

Based on the previous correlation heatmap, 'GarageYrBlt' is highly correlated with 'YearBuilt', so let's replace the missing values by medians of 'YearBuilt'. To do that, we need to cut 'YearBuilt' into sections since it is a numeric variable

In [ ]:
# Cut 'YearBuilt' into 10 parts
dataFull['YearBuiltCut'] = pd.qcut(dataFull.YearBuilt,10)
# Impute the missing values of 'GarageYrBlt' based on the median of 'YearBuilt' 
dataFull['GarageYrBlt']= dataFull.groupby(['YearBuiltCut'])['GarageYrBlt'].transform(lambda x : x.fillna(x.median()))
# convert the values to integers
dataFull['GarageYrBlt'] = dataFull['GarageYrBlt'].astype(int)
# Drop 'YearBuiltCut' column
dataFull.drop('YearBuiltCut',axis=1,inplace=True)
# Check if missing values are imputed successfully
dataFull.isnull().sum()[dataFull.isnull().sum()>0].sort_values(ascending = False)

Based on the previous correlation heatmap, 'LotFrontage' is highly correlated with 'LotArea' and 'LotFrontage'. So let's use the same method to fill the missing values

In [ ]:
# Cut 'LotArea' into 10 parts
dataFull['LotAreaCut'] = pd.qcut(dataFull.LotArea,10)
# Impute the missing values of 'LotFrontage' based on the median of 'LotArea' and 'Neighborhood'
dataFull['LotFrontage']= dataFull.groupby(['LotAreaCut','Neighborhood'])['LotFrontage'].transform(lambda x : x.fillna(x.median()))
dataFull['LotFrontage']= dataFull.groupby(['LotAreaCut'])['LotFrontage'].transform(lambda x : x.fillna(x.median()))
# Drop 'LotAreaCut' column
dataFull.drop('LotAreaCut',axis=1,inplace=True)

# Check if missing values are imputed successfully
dataFull.isnull().sum()[dataFull.isnull().sum()>0].sort_values(ascending = False)

The only missing values that are left are within SalePrice, which is exactly the number of lignes in the test data (the values that we need to predict)

In [ ]:
%%HTML
<h1>Correcting Features</h1>

If we take a look at the numeric variables, we see that some of them obviously don't make a sense being numerical like year related features. Let's take a closer look at each one of them in the data description file and see which ones need to be converted to categorical type.

In [ ]:
dataFull.select_dtypes(include=[np.number]).columns

In [ ]:
# Converting numeric features to categorical features
strCols = ['YrSold','YearRemodAdd','YearBuilt','MoSold','MSSubClass','GarageYrBlt']
for i in strCols:
    dataFull[i]=dataFull[i].astype(str)

In [ ]:
%%HTML
<h1>Adding Features</h1>

First, we will map some categorical variable that represent some sort of rating to an integer score

In [ ]:
dataFull.select_dtypes(include=[np.object]).columns

In [ ]:
dataFull["oExterQual"] = dataFull.ExterQual.map({'Fa':1, 'TA':2, 'Gd':3, 'Ex':4})
dataFull["oBsmtQual"] = dataFull.BsmtQual.map({'None':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})
dataFull["oBsmtExposure"] = dataFull.BsmtExposure.map({'None':1, 'No':2, 'Av':3, 'Mn':3, 'Gd':4})
dataFull["oHeatingQC"] = dataFull.HeatingQC.map({'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})
dataFull["oKitchenQual"] = dataFull.KitchenQual.map({'Fa':1, 'TA':2, 'Gd':3, 'Ex':4})
dataFull["oFireplaceQu"] = dataFull.FireplaceQu.map({'None':1, 'Po':2, 'Fa':3, 'TA':4, 'Gd':5, 'Ex':6})
dataFull["oGarageFinish"] = dataFull.GarageFinish.map({'None':1, 'Unf':2, 'RFn':3, 'Fin':4})
dataFull["oPavedDrive"] = dataFull.PavedDrive.map({'N':1, 'P':2, 'Y':3})

Next, we will add up some numeric features with each other to create new features that make sense

In [ ]:
dataFull.select_dtypes(include=[np.number]).columns

In [ ]:
dataFull['HouseSF'] = dataFull['1stFlrSF'] + dataFull['2ndFlrSF'] + dataFull['TotalBsmtSF']
dataFull['PorchSF'] = dataFull['3SsnPorch'] + dataFull['EnclosedPorch'] + dataFull['OpenPorchSF'] + dataFull['ScreenPorch']
dataFull['TotalSF'] = dataFull['HouseSF'] + dataFull['PorchSF'] + dataFull['GarageArea']

In [ ]:
%%HTML
<h1>Skewness and Kurtosis</h1>

In [ ]:
# Estimate Skewness and Kurtosis of the data
trainData.skew(), trainData.kurt()

In [ ]:
# Plot the Skewness of the data
sns.distplot(trainData.skew(),axlabel ='Skewness')

In [ ]:
# Plot the Kurtosis of the data
sns.distplot(trainData.kurt(),axlabel ='Kurtosis')

There isn't much kurtosis in the data columns, but Skewness is very present, meaning that distribution is not symetrical

In [ ]:
%%HTML
<h1>Label Encoding</h1>

For this section we will use Pipelines which are a way to streamline a lot of the routine processes. It provides a way to take code, fit it to the training data, apply it to the test data without having to copy and paste everything.
We will create three classes : first for label encoding, second for skewness, and third for one hot label encoding.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.preprocessing import LabelEncoder, Imputer
from scipy.stats import skew

# Label encoding class
class labenc(BaseEstimator,TransformerMixin):
    def __init__(self):
        pass
    def fit(self,X,y=None):
        return self
    def transform(self,X):
        label = LabelEncoder()
        X['YrSold']=label.fit_transform(X['YrSold'])
        X['YearRemodAdd']=label.fit_transform(X['YearRemodAdd'])
        X['YearBuilt']=label.fit_transform(X['YearBuilt'])
        X['MoSold']=label.fit_transform(X['MoSold'])
        X['GarageYrBlt']=label.fit_transform(X['GarageYrBlt'])
        return X
    
# Skewness transform class
class skewness(BaseEstimator,TransformerMixin):
    def __init__(self):
        pass
    def fit(self,X,y=None):
        return self
    def transform(self,X):
        skewness = X.select_dtypes(include=[np.number]).apply(lambda x: skew(x))
        skewness_features = skewness[abs(skewness) >= 1].index
        X[skewness_features] = np.log1p(X[skewness_features])
        return X

# One hot encoding class
class onehotenc(BaseEstimator,TransformerMixin):
    def __init__(self):
        pass
    def fit(self,X,y=None):
        return self
    def transform(self,X):
        X = pd.get_dummies(X)
        return X

In [ ]:
# Creating a copy of the full dataset
dataFullCopy = dataFull.copy()

# Creating a new fata with aplied transformations using sklearn Pipeline
from sklearn.pipeline import Pipeline
pipeline = Pipeline([('labenc',labenc()),('skewness',skewness()),('onehotenc',onehotenc())])
dataPipeline = pipeline.fit_transform(dataFullCopy)
dataFull.shape, dataPipeline.shape

We can see now that the number of features increases from 88 to 328

In [ ]:
dataPipeline.head()

Now we split the data to training and testing datasets

In [ ]:

X_train = dataPipeline[:trainData.shape[0]]
y_train = X_train['SalePrice']
X_train.drop(columns = 'SalePrice', inplace=True)
X_test = dataPipeline[trainData.shape[0]:]
X_test.drop(columns = 'SalePrice', inplace=True)
X_train.shape, y_train.shape, X_test.shape

In [ ]:
%%HTML
<h1>Transformation and Scaling</h1>

In [ ]:
# SalesPrices plot with three different fitted distributions
plt.figure(2); plt.title('Normal')
sns.distplot(y_train, kde=False, fit=st.norm)
plt.figure(3); plt.title('Log Normal')
sns.distplot(y_train, kde=False, fit=st.lognorm)
plt.figure(1); plt.title('Johnson SU')
sns.distplot(y_train, kde=False, fit=st.johnsonsu)

Normal distribution doesn't fit, so SalePrice need to be transformed before creating the model. Best fit is unbounded Johnson distribution, altough log normal distribution also fits well

In [ ]:
# transforming 'SalePrice' into normal distribution
y_train_transformed = np.log(y_train)
y_train_transformed.skew(), y_train_transformed.kurt()

In [ ]:
# plotting 'SalePrice' before and after the transformation
plt.figure(1); plt.title('Before transformation')
sns.distplot(y_train)
plt.figure(2); plt.title('After transformation')
sns.distplot(y_train_transformed)

In [ ]:
# Using Robust Scaler to transform X_train
from sklearn.preprocessing import RobustScaler
robust_scaler = RobustScaler()
X_train_scaled = robust_scaler.fit(X_train).transform(X_train)
X_test_scaled = robust_scaler.transform(X_test)

In [ ]:
# Shape of final data we will be working on
X_train_scaled.shape, y_train_transformed.shape, X_test_scaled.shape

In [ ]:
%%HTML
<h1>Feature Selection</h1>

We will use lasso regression (l1 regularization method). Lasso shrinks the less important feature’s coefficient to zero thus, removing some feature altogether. We can also use it to find the most important features in our dataset.

In [ ]:
# Display features by their importance (lasso regression coefficient)
from sklearn.linear_model import Lasso
lasso = Lasso(alpha = 0.001)
lasso.fit(X_train_scaled,y_train_transformed)
y_pred_lasso = lasso.predict(X_test_scaled)
lassoCoeff = pd.DataFrame({"Feature Importance":lasso.coef_}, index=dataPipeline.drop(columns = 'SalePrice').columns)
lassoCoeff.sort_values("Feature Importance",ascending=False)

In [ ]:
# Plot features by importance (feature coefficient in the model)
lassoCoeff[lassoCoeff["Feature Importance"]!=0].sort_values("Feature Importance").plot(kind="barh",figsize=(20,35))

What's intersting here is that two of the variables that we have created 'HouseSF' and 'PorchSF' perform actually bad compared to their components. But when we sum all the surfaces as in 'TotalSF', which is just a combination of features that are significantly unimportant in this model, we suddently obtain the most important feature in the dataset.

In [ ]:
%%HTML
<h1>Principal Component Analysis</h1>

PCA is used to decompose a multivariate dataset in a set of successive orthogonal components that explain a maximum amount of the variance

In [ ]:
from sklearn.decomposition import PCA
# Concatenate the training and test datasets into a single datafram
dataFull2 = np.concatenate([X_train_scaled,X_test_scaled])
# Choose the number of principle components such that 95% of the variance is retained
pca = PCA(0.95)
dataFull2 = pca.fit_transform(dataFull2)
varPCA = np.round(pca.explained_variance_ratio_*100, decimals = 1)
# Principal Component Analysis of data
print(varPCA)

In [ ]:
# Principal Component Analysis plot of the data
plt.figure(figsize=(16,12))
plt.bar(x=range(1,len(varPCA)+1), height = varPCA)
plt.ylabel("Explained Variance (%)", size = 15)
plt.xlabel("Principle Components", size = 15)
plt.title("Principle Component Analysis Plot : Training Data", size = 15)
plt.show()

In [ ]:
# Shape of final data we will be working on
X_train_scaled = dataFull2[:trainData.shape[0]]
X_test_scaled = dataFull2[trainData.shape[0]:]
X_train_scaled.shape, y_train_transformed.shape, X_test_scaled.shape

We see that now we have 87 features instead of the 327 features that we had before using PCA.

In [ ]:
%%HTML
<h1>Testing Different Models</h1>

Now that we have finished preparing our data, it's time to test different models to see which one performs the best.
The models we will be testing are : 
- Linear regression
- Support vector regression
- Stochastic gradient descent
- Gradient boosting tree
- Random forest
- Lasso regression
- Ridge regression
- Elastic net regularization
- Extra trees regression

In [ ]:
# importing the models
from sklearn.linear_model import LinearRegression, BayesianRidge, ElasticNet, Lasso, SGDRegressor, Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import ExtraTreesRegressor,GradientBoostingRegressor,RandomForestRegressor
from sklearn.svm import LinearSVR,SVR
# creating the models
models = [
             LinearRegression(),
             SVR(),
             SGDRegressor(),
             SGDRegressor(max_iter=1000, tol = 1e-3),
             GradientBoostingRegressor(),
             RandomForestRegressor(),
             Lasso(),
             Lasso(alpha=0.01,max_iter=10000),
             Ridge(),
             BayesianRidge(),
             KernelRidge(),
             KernelRidge(alpha=0.6,kernel='polynomial',degree = 2,coef0=2.5),
             ElasticNet(),
             ElasticNet(alpha = 0.001,max_iter=10000),    
             ExtraTreesRegressor(),
             ]

names = ['Linear regression','Support vector regression','Stochastic gradient descent','Stochastic gradient descent 2','Gradient boosting tree','Random forest','Lasso regression','Lasso regression 2','Ridge regression','Bayesian ridge regression','Kernel ridge regression','Kernel ridge regression 2','Elastic net regularization','Elastic net regularization 2','Extra trees regression']




In [ ]:
# Define a root mean square error function
def rmse(model,X,y):
    rmse = np.sqrt(-cross_val_score(model,X,y,scoring="neg_mean_squared_error",cv=5))
    return rmse

In [ ]:
from sklearn.model_selection import KFold,cross_val_score
warnings.filterwarnings('ignore')

# Perform 5-folds cross-calidation to evaluate the models 
for model, name in zip(models, names):
    # Root mean square error
    score = rmse(model,X_train_scaled,y_train_transformed)
    print("- {} : mean : {:.6f}, std : {:4f}".format(name, score.mean(),score.std()))

Surprisingly, the Random forest and Extra trees regression models are the ones who performed the worst, and the linear regression model performed actually pretty good relative to the other models.
By compiling the above code several times and observing the different scores each time, we can classify the models by accuracy :

- 1st : Kernel ridge regression
- 2nd : Elastic net regularization and Bayesian ridge regression
- 3rd : Ridge regression and Linear regression
- 4rth : Support vector regression
- 5th : Gradient boosting tree
- 6th : Stochastic gradient  and Lasso regression
- 7th : Random forest and Extra trees regression

I think we got a good score in Elastic net regularization, Lasso regression and Stochastic gradient descent because we chose some good parameters. We can see that their score above is very bad when not specifing parameter values. So if we really want to know to best model, we need to choose optimal parameters for all the models, and tha's what we will do in the next section.

In [ ]:
%%HTML
<h1>Hyper-parameter Tuning</h1>

For choosing the most optimal hyper-parameters, we will perform gird search. the class GridSearchCV exhaustively considers all parameter combinations and generates candidates from a grid of parameter values specified with the param_grid parameter.
Since we will use the same procedure for all models, we will start by creating a function which takes specified parameter values as entry.

In [ ]:
from sklearn.model_selection import GridSearchCV

class gridSearch():
    def __init__(self,model):
        self.model = model
    def grid_get(self,param_grid):
        grid_search = GridSearchCV(self.model,param_grid,cv=5,scoring='neg_mean_squared_error')
        grid_search.fit(X_train_scaled,y_train_transformed)
        grid_search.cv_results_['mean_test_score'] = np.sqrt(-grid_search.cv_results_['mean_test_score'])
        print(pd.DataFrame(grid_search.cv_results_)[['params','mean_test_score','std_test_score']])
        print('\nBest parameters : {}, best score : {}'.format(grid_search.best_params_,np.sqrt(-grid_search.best_score_)))

1. Kernel ridge regression

In [ ]:
gridSearch(KernelRidge()).grid_get(
        {'alpha':[3.5,4,4.5,5,5.5,6,6.5], 'kernel':["polynomial"], 'degree':[3],'coef0':[1,1.5,2,2.5,3,3.5]})

2. Elastic net regularization

In [ ]:
gridSearch(ElasticNet()).grid_get(
        {'alpha':[0.006,0.0065,0.007,0.0075,0.008],'l1_ratio':[0.070,0.075,0.080,0.085,0.09,0.095],'max_iter':[10000]})

3. Ridge regression

In [ ]:
gridSearch(Ridge()).grid_get(
        {'alpha':[10,20,25,30,35,40,45,50,55,57,60,65,70,75,80,100],'max_iter':[10000]})

4. Support vector regression

In [ ]:
gridSearch(SVR()).grid_get(
        {'C':[13,15,17,19,21],'kernel':["rbf"],"gamma":[0.0005,0.001,0.002,0.01],"epsilon":[0.01,0.02,0.03,0.1]})

5. Lasso regression

In [ ]:
gridSearch(Lasso()).grid_get(
       {'alpha':[0.01,0.001,0.0001,0.0002,0.0003,0.0004,0.0005,0.0006,0.0007,0.0008,0.0009],'max_iter':[10000]})

In [ ]:
lasso = Lasso(alpha= 0.0006, max_iter= 10000)
ridge = Ridge(alpha=35, max_iter= 10000)
svr = SVR(C = 13, epsilon= 0.03, gamma = 0.001, kernel = 'rbf')
ker = KernelRidge(alpha=6.5 ,kernel='polynomial', degree=3 , coef0=2.5)
ela = ElasticNet(alpha=0.007,l1_ratio=0.07,max_iter=10000)
bay = BayesianRidge()

In [ ]:
%%HTML
<h1>Combining Models</h1>

In order to further improve our model accuracy. We will use an ensemble method. I chose to use stacking. Stacked generalization is an ensemble method where the models are combined using another machine learning algorithm. The basic idea is to train machine learning algorithms with training dataset and then generate a new dataset with these models. Then this new dataset is used as input for the combiner machine learning algorithm.

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor,GradientBoostingRegressor,RandomForestRegressor

In [ ]:
# Creating the stacking function
class stacking(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self,mod,meta_model):
        self.mod = mod
        self.meta_model = meta_model
        kff = KFold(n_splits=5, random_state=42, shuffle=True)
        self.kf = kff
        
    def fit(self,X,y):
        self.saved_model = [list() for i in self.mod]
        oof_train = np.zeros((X.shape[0], len(self.mod)))
        
        for i,model in enumerate(self.mod):
            for train_index, val_index in self.kf.split(X,y):
                renew_model = clone(model)
                renew_model.fit(X[train_index], y[train_index])
                self.saved_model[i].append(renew_model)
                oof_train[val_index,i] = renew_model.predict(X[val_index])
        
        self.meta_model.fit(oof_train,y)
        return self
    
    def predict(self,X):
        whole_test = np.column_stack([np.column_stack(model.predict(X) for model in single_model).mean(axis=1) 
                                      for single_model in self.saved_model]) 
        return self.meta_model.predict(whole_test)
    
    def get_oof(self,X,y,test_X):
        oof = np.zeros((X.shape[0],len(self.mod)))
        test_single = np.zeros((test_X.shape[0],5))
        test_mean = np.zeros((test_X.shape[0],len(self.mod)))
        for i,model in enumerate(self.mod):
            for j, (train_index,val_index) in enumerate(self.kf.split(X,y)):
                clone_model = clone(model)
                clone_model.fit(X[train_index],y[train_index])
                oof[val_index,i] = clone_model.predict(X[val_index])
                test_single[:,j] = clone_model.predict(test_X)
            test_mean[:,i] = test_single.mean(axis=1)
        return oof, test_mean

In [ ]:
# Impute the training dataset
X_scaled_imputed = Imputer().fit_transform(X_train_scaled)
y_log_imputed = Imputer().fit_transform(y_train_transformed.values.reshape(-1,1)).ravel()

In [ ]:
X_scaled_imputed.shape,y_log_imputed.shape,X_test_scaled.shape

In [ ]:
# Calculating the score
stack_model = stacking(mod=[lasso,ridge,svr,ker,ela,bay],meta_model=ker)
score = rmse(stack_model,X_scaled_imputed,y_log_imputed)
print(score.mean())

We obtain a score of 0.113, which is slightly better than the average score of the other models

In [ ]:
# Combining the extracted features generated from stacking whith original features
X_train_stack,X_test_stack = stack_model.get_oof(X_scaled_imputed,y_log_imputed,X_test_scaled)
X_train_add = np.hstack((X_scaled_imputed,X_train_stack))
X_test_add = np.hstack((X_test_scaled,X_test_stack))
X_train_add.shape,X_test_add.shape

In [ ]:
# Calculate the final score of the model
score = rmse(stack_model,X_train_add,y_log_imputed)
print(score.mean())

The last score we obtain is 0.1074, which is quite good

In [ ]:
%%HTML
<h1>Making Predictions</h1>

Now it's time to make predictions and store them in a csv file with corresponding Ids. after we make prediction we need to transform them to their original shape with exponential function

In [ ]:
# Fit the model to the dataset generated with stacking
stack_model.fit(X_train_add,y_log_imputed)

In [ ]:
# Making prediction on the test set generated by stacking
predicted_prices = np.exp(stack_model.predict(X_test_add))
# Prepare the csv file
my_submission = pd.DataFrame({'Id': testData.Id, 'SalePrice': predicted_prices})
my_submission.to_csv('submission.csv', index=False)

The last time I submitted, I was the 859th (top 20%) with a score of 0.11934

In [ ]:
%%HTML
<h2>Thanks for reading my notebook. If you liked my kernel please kindly UPVOTE for other people to see. If you have any remarks, please leave a comment bellow.<h2>